In [ ]:
import numpy as np
import pandas as pd
import pymysql
import re
import regex
import random

In [ ]:
#нахождение типов действий, являющихся маркерами информации о беременности

con = pymysql.connect(host='localhost', port=3303,user='root',password='RootPassword', database='newschema')
with con: 
    cur = con.cursor()
    cur.execute("SELECT distinct id FROM actiontype WHERE group_id in (695,692)")
    rows = cur.fetchall()
    t= np.array(rows)
    t1=[]
    for i in t:
        t1.append(i[0])

In [ ]:
#все женщины

con = pymysql.connect(host='localhost', port=3303,user='root',password='RootPassword', database='newschema') 
with con: 
    cur = con.cursor()
    #нахождение id и даты рождения всех женщин
    cur.execute("SELECT id,birthDate,notes FROM client WHERE sex=2")
    rows = cur.fetchall()
    women = {}
    woman= np.array(rows)
    w1=[] 
    w2=[]
    w3=[]
    for i in woman:
        w1.append(i[0])#лист id
        w2.append(i[1])
        w3.append(i[2])
    women['id']=w1
    women['Date']=w2
    women['Notes']=w3
    df_women = pd.DataFrame(women) #dataframe всех женщин

In [ ]:
#нахождение id клиентов, имеющих записи с найденными выше типами действий, предположительно относящимся к беременности

con = pymysql.connect(host='localhost', port=3303,user='root',password='RootPassword', database='newschema')
with con: 
    cur = con.cursor()
    cur.execute("SELECT distinct client_id from event INNER JOIN action ON action.event_id=event.id WHERE action.actionType_id IN (705, 706, 23944, 23955, 23956, 23957, 46626, 50028, 62365, 62395, 65975, 65977, 65981, 65985, 66014, 66023, 66026, 66027, 66542, 66543, 66545, 73373, 73056)")
    rows = cur.fetchall()
    r= np.array(rows)
    r1=[]
    for i in r:
        r1.append(i[0])
        
bl1=r1
random.shuffle(bl1)

In [ ]:
#Сбор данных, нужно менять только номер типа действия

dictionaryData = {
    'id' : [],
    'action_id' : [],
    'age': [],
    'weeks': [],
    'com': [],
    'diagnos': [],
    'value' : [],
    'ill':[],
    'heredity':[],
    'habits' : [],
    'gynecology' : [],
    'inspection': [],
    'husband': [],
    'course ': [],
    'pills': [],
    'recommendations': [],
    'previous' : [],
    'fetus': []
}
df_value=pd.DataFrame(dictionaryData)


for j in range(398):
    bl=bl1[j*100:100+j*100] 
    con = pymysql.connect(host='localhost', port=3303,user='root',password='RootPassword', database='newschema')
    print(j)
    with con: 
 
        cur = con.cursor()
        for ide in bl:
            cur.execute("select birthDate, createDatetime from client where id =%s", int(ide))            
            rows = cur.fetchall()
            r= np.array(rows)
            if r[0][1] == '0000-00-00 00:00:00':
                print('ide')
                continue
            e=r[0][0]
            b=r[0][1].date()
            c=int((b-e).days/365)
            
            cur.execute("SELECT action.id, actionproperty_string.value, actionpropertytype.name FROM actionproperty_string INNER JOIN actionproperty ON actionproperty.id=actionproperty_string.id INNER JOIN actionpropertytype ON actionpropertytype.id=actionproperty.type_id INNER JOIN action ON action.id=actionproperty.action_id INNER JOIN actiontype ON actiontype.id=action.actionType_id INNER JOIN event ON event.id=action.event_id WHERE client_id=%s AND actiontype.id IN (23944)", int(ide))
            rows = cur.fetchall()
            r= np.array(rows)
            if len(r)!=0:
                a=[]
                com=[]
                di=[]
                ill=[]
                her=[] 
                hab= []
                gynecology=[]
                inspection=[]
                husband=[]
                course=[]
                pills=[]
                recommendations= []
                previous=[]
                fetus=[]
                act_id=r[0][0]
                for i in r:
                    if act_id==i[0]:
                        if i[2]==('Жалобы')or i[2]==('ДЛЯ ОТЧЕТА: Обращение')or i[2]==('Жалобы гинекологические (г)'):              
                            com.append(i[1])
                        else:                                                 
                            if i[2]==('Диагноз')or i[2]==('Диагноз (Гестоз)')or i[2]==('Клинический диагноз')or i[2]==('Предварительный диагноз')or i[2]==('Диагноз (ФНП)')or i[2]==('ДЛЯ ОТЧЕТА: Совпадение диагнозов')or i[2]==('ДЛЯ ОТЧЕТА: Заключительный диагноз'):              
                                di.append(i[1])
                            else:
                                if i[2]==('Перенесенные заболевания в детстве')or i[2]==('Анамнез жизни')or i[2]==('Отягощенный анамнез')or i[2]==('Травмы')or i[2]==('Перенесенные заболевания взрослой'):             
                                    ill.append(i[1])
                                else:
                                    if i[2]==('Тромботический анамнез')or i[2]==('Сахарный диабет у родственников')or i[2]==('Наследственность')or i[2]==('Пороки развития в семье')or i[2]==('Ранние инфаркты, инсульты, тромбозы у родственников')or i[2]==('Случаи рождения детей с ВПР в семье'):             
                                        her.append(i[1])
                                    else:
                                        if i[2]==('Вредные привычки'):             
                                            hab.append(i[1])
                                        else:
                                            if i[2]==('Здоровье мужа')or i[2]==('Сведения о муже'):             
                                                husband.append(i[1])
                                            else:
                                                if i[2]==('ДЛЯ ОТЧЕТА: ОАГА')or i[2]==('ДЛЯ ОТЧЕТА: Акушерские осложнения') or i[2]==('Инф. гепатиты')or i[2]==('Вен. заболевания')or i[2]==('Предменструальный период (г)')or i[2]==('Менструация')or i[2]==('Последняя менструация')or i[2]==('Бесплодие')or i[2]==('СПИД (ВИЧ)')or i[2]==('Гинекологические заболевания и операции')or i[2]==('ИППП и другие заболевания')or i[2]==('Гинекологический статус'):             
                                                    gynecology.append(i[1])
                                                else:
                                                    if i[2]==('Живот при пальпации')or i[2]==('Живот')or i[2]==('Вены')or i[2]==('Особенности телосложения')or i[2]==('Состояние молочных желез')or i[2]==('Молочные железы')or i[2]==('Кожа')or i[2]==('Кожные покровы') or i[2]==('Кожа и видимые слизистые') or i[2]==('Слизистые')or i[2]==('Лимфоузлы')or i[2]==('Ладони и стопы')or i[2]==('Наружное акушерское исследование: матка в')or i[2]==('Наружное акушерское исследование: матка') or i[2]==('Матка при осмотре')or i[2]==('Наружные половые органы')or i[2]==('В зеркалах: влагалище')or i[2]==('Влагалища') or i[2]==('Влагалищное исследование') or i[2]==('В зеркалах шейка матки') or i[2]==('Шейка матки бимануально')or i[2]==('Слизистая шейки матки')or i[2]==('Синдром "зрачка"')or i[2]==('шейка матки')or i[2]==('выделения')or i[2]==('Выделения')or i[2]==('Выделения из половых путей')or i[2]==('Бимануально: тело матки')or i[2]==('Бимануально тело матки')or i[2]==('Придатки')or i[2]==('Оволосение')or i[2]==('Своды')or i[2]==('Мыс'):             
                                                        inspection.append(i[1]) 
                                                    else:
                                                        if i[2]==('ДЛЯ ОТЧЕТА: Без показаний:')or i[2]==('ДЛЯ ОТЧЕТА: Без показаний') or i[2]==('Особенности течения настоящей  беременности')or i[2]==('Течение беременности')or i[2]==('Общее состояние')or i[2]==('Особенности беременности')or i[2]==('Осложнения беременности:')or i[2]==('Данные дополнительных методов исследования'):             
                                                            course.append(i[1])
                                                        else:
                                                            if i[2]==('В настоящее время принимает')or i[2]==('Медикаменты принимаемые в настоящее время:') or i[2]==('В настоящее время принимает со слов беременной') or  i[2]==('В настоящее время принимает со слов пациентки'):              
                                                                pills.append(i[1])
                                                            else:
                                                                if i[2]==('Рекомендации')or i[2]==('ДЛЯ ОТЧЕТА: Рекомендации'):              
                                                                    recommendations.append(i[1])
                                                                else:
                                                                    if i[2]==('ДЛЯ ОТЧЕТА: Паритет:')or i[2]==('Исход предыдущих беременностей')or i[2]==('Сведения о предыдущих беременностях. 1.№')or i[2]==('Исход предыдущих беременностей ст'):              
                                                                        previous.append(i[1])
                                                                    else:
                                                                        if i[2]==('ДЛЯ ОТЧЕТА: Проведенные УЗ исследования')or i[2]==('Сердцебиение плода')or i[2]==('Положение плода')or i[2]==('Шевеление плода')or i[2]==('Где находится')or i[2]==('С/биение плода')or i[2]==('позиция')or i[2]==('вид позиции')or i[2]==('Предлежит')or i[2]==('Шевеление'):              
                                                                            fetus.append(i[1])
                                                                        else:
                                                                            a.append(i[1])
                    else:
                        d=len(df_value)
                        t=(" ".join(a))
                        t1=(" ".join(com))
                        t2=(" ".join(di))
                        t3=(" ".join(ill))
                        t4=(" ".join(her))
                        t5=(" ".join(hab))
                        t6=(" ".join(gynecology))
                        t7=(" ".join(inspection))
                        t8=(" ".join(husband))
                        t9=(" ".join(course))
                        t10=(" ".join(pills))
                        t11=(" ".join(recommendations))
                        t12=(" ".join(previous))
                        t13=(" ".join(fetus))
                        match = regex.search(r'Беременность\D*(?P<num>(\d+|(\d+.\d+)))\s*нед',t2)
                        if match:
                            match1= re.search(r'(?P<num>(\d+))',match.group('num'))
                            week=int(match1.group('num'))
                        else:
                            week=0
                        df_value.loc[d] = [ide,act_id,c,week,t1,t2,t,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13]
                        #сохранили предыдущие значения, сохраняем первую строчку
                        
                        act_id=i[0]
                        a=[]
                        t=[]
                        com=[]
                        di=[]
                        ill=[]
                        her=[] #наследственность
                        hab= []
                        gynecology=[]
                        inspection=[]
                        husband=[]
                        course=[]
                        pills=[]
                        recommendations= []
                        previous=[]
                        fetus=[]
                        t1=[]
                        t2=[]
                        t3=[]
                        t4=[]
                        t5=[]
                        t6=[]
                        t7=[]
                        t8=[]
                        t9=[]
                        t10=[]
                        t11=[]
                        t12=[]
                        t13=[]
                        if i[2]==('Жалобы')or i[2]==('ДЛЯ ОТЧЕТА: Обращение')or i[2]==('Жалобы гинекологические (г)'):              
                            com.append(i[1])
                        else:                                                 
                            if i[2]==('Диагноз')or i[2]==('Диагноз (Гестоз)')or i[2]==('Клинический диагноз')or i[2]==('Предварительный диагноз')or i[2]==('Диагноз (ФНП)')or i[2]==('ДЛЯ ОТЧЕТА: Совпадение диагнозов')or i[2]==('ДЛЯ ОТЧЕТА: Заключительный диагноз'):              
                                di.append(i[1])
                            else:
                                if i[2]==('Перенесенные заболевания в детстве')or i[2]==('Анамнез жизни')or i[2]==('Отягощенный анамнез')or i[2]==('Травмы')or i[2]==('Перенесенные заболевания взрослой'):             
                                    ill.append(i[1])
                                else:
                                    if i[2]==('Тромботический анамнез')or i[2]==('Сахарный диабет у родственников')or i[2]==('Наследственность')or i[2]==('Пороки развития в семье')or i[2]==('Ранние инфаркты, инсульты, тромбозы у родственников')or i[2]==('Случаи рождения детей с ВПР в семье'):             
                                        her.append(i[1])
                                    else:
                                        if i[2]==('Вредные привычки'):             
                                            hab.append(i[1])
                                        else:
                                            if i[2]==('Здоровье мужа')or i[2]==('Сведения о муже'):             
                                                husband.append(i[1])
                                            else:
                                                if i[2]==('ДЛЯ ОТЧЕТА: ОАГА')or i[2]==('ДЛЯ ОТЧЕТА: Акушерские осложнения') or i[2]==('Инф. гепатиты')or i[2]==('Вен. заболевания')or i[2]==('Предменструальный период (г)')or i[2]==('Менструация')or i[2]==('Последняя менструация')or i[2]==('Бесплодие')or i[2]==('СПИД (ВИЧ)')or i[2]==('Гинекологические заболевания и операции')or i[2]==('ИППП и другие заболевания')or i[2]==('Гинекологический статус'):             
                                                    gynecology.append(i[1])
                                                else:
                                                    if i[2]==('Живот при пальпации')or i[2]==('Живот')or i[2]==('Вены')or i[2]==('Особенности телосложения')or i[2]==('Состояние молочных желез')or i[2]==('Молочные железы')or i[2]==('Кожа')or i[2]==('Кожные покровы') or i[2]==('Кожа и видимые слизистые') or i[2]==('Слизистые')or i[2]==('Лимфоузлы')or i[2]==('Ладони и стопы')or i[2]==('Наружное акушерское исследование: матка в')or i[2]==('Наружное акушерское исследование: матка') or i[2]==('Матка при осмотре')or i[2]==('Наружные половые органы')or i[2]==('В зеркалах: влагалище')or i[2]==('Влагалища') or i[2]==('Влагалищное исследование') or i[2]==('В зеркалах шейка матки') or i[2]==('Шейка матки бимануально')or i[2]==('Слизистая шейки матки')or i[2]==('Синдром "зрачка"')or i[2]==('шейка матки')or i[2]==('выделения')or i[2]==('Выделения')or i[2]==('Выделения из половых путей')or i[2]==('Бимануально: тело матки')or i[2]==('Бимануально тело матки')or i[2]==('Придатки')or i[2]==('Оволосение')or i[2]==('Своды')or i[2]==('Мыс'):             
                                                        inspection.append(i[1]) 
                                                    else:
                                                        if i[2]==('ДЛЯ ОТЧЕТА: Без показаний:')or i[2]==('ДЛЯ ОТЧЕТА: Без показаний') or i[2]==('Особенности течения настоящей  беременности')or i[2]==('Течение беременности')or i[2]==('Общее состояние')or i[2]==('Особенности беременности')or i[2]==('Осложнения беременности:')or i[2]==('Данные дополнительных методов исследования'):             
                                                            course.append(i[1])
                                                        else:
                                                            if i[2]==('В настоящее время принимает')or i[2]==('Медикаменты принимаемые в настоящее время:') or i[2]==('В настоящее время принимает со слов беременной') or  i[2]==('В настоящее время принимает со слов пациентки'):              
                                                                pills.append(i[1])
                                                            else:
                                                                if i[2]==('Рекомендации')or i[2]==('ДЛЯ ОТЧЕТА: Рекомендации'):              
                                                                    recommendations.append(i[1])
                                                                else:
                                                                    if i[2]==('ДЛЯ ОТЧЕТА: Паритет:')or i[2]==('Исход предыдущих беременностей')or i[2]==('Сведения о предыдущих беременностях. 1.№')or i[2]==('Исход предыдущих беременностей ст'):              
                                                                        previous.append(i[1])
                                                                    else:
                                                                        if i[2]==('ДЛЯ ОТЧЕТА: Проведенные УЗ исследования')or i[2]==('Сердцебиение плода')or i[2]==('Положение плода')or i[2]==('Шевеление плода')or i[2]==('Где находится')or i[2]==('С/биение плода')or i[2]==('позиция')or i[2]==('вид позиции')or i[2]==('Предлежит')or i[2]==('Шевеление'):              
                                                                            fetus.append(i[1])
                                                                        else:
                                                                            a.append(i[1])
                    
                d=len(df_value)           
                t=(" ".join(a))
                t1=(" ".join(com))
                t2=(" ".join(di))
                t3=(" ".join(ill))
                t4=(" ".join(her))
                t5=(" ".join(hab))
                t6=(" ".join(gynecology))
                t7=(" ".join(inspection))
                t8=(" ".join(husband))
                t9=(" ".join(course))
                t10=(" ".join(pills))
                t11=(" ".join(recommendations))
                t12=(" ".join(previous))
                t13=(" ".join(fetus))
                match = regex.search(r'Беременность\D*(?P<num>(\d+|(\d+.\d+)))\s*нед',t2)
                if match:
                    match1= re.search(r'(?P<num>(\d+))',match.group('num'))
                    week=int(match1.group('num'))
                else:
                    week=0
                df_value.loc[d] = [ide,act_id,c,week,t1,t2,t,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13]
                a=[]
                t=[]
                com=[]
                di=[]
                ill=[]
                her=[]
                hab= []
                gynecology=[]
                inspection=[]
                husband=[]
                course=[]
                pills=[]
                recommendations= []
                previous=[]
                fetus=[]
                t1=[]
                t2=[]
                t3=[]
                t4=[]
                t5=[]
                t6=[]
                t7=[]
                t8=[]
                t9=[]
                t10=[]
                t11=[]
                t12=[]
                t13=[]

In [ ]:
df_value1=df_value
df_value1.to_csv('1_type.csv')